In [1]:
import requests
import json

In [2]:
app_id = "d5fd616b"
app_key = "5c06fb4f5bd545920f9d620b5a5a63d4"

Given the food name, print the calories.

In [7]:
url = "https://trackapi.nutritionix.com/v2/natural/nutrients"
headers = {
    "Content-Type": "application/json",
    "x-app-id": app_id,
    "x-app-key": app_key
}
body = {
    "query": "grape"
}

response = requests.post(url, headers=headers, data=json.dumps(body))

for food in response.json()["foods"]:
    print(food["food_name"] + ": " + str(food["nf_calories"]) + " calories")

grape: 33.81 calories


Autocomplete the given query.

In [9]:
query = "hamburger"

url = "https://trackapi.nutritionix.com/v2/search/instant/?query=" + query
headers = {
    "Content-Type": "application/json",
    "x-app-id": app_id,
    "x-app-key": app_key
}

response = requests.get(url, headers=headers)

if response.status_code != 200:
    raise Exception(f"Error: {response.status_code}")
else:
    for common in response.json()["common"]:
        print(common["food_name"])
    for branded in response.json()["branded"]:
        print(branded["food_name"])


hamburger
hamburgers
hamburger bun
soy hamburger
hamburger meat
lean hamburger
hamburger taco
hamburger buns
hamburger soup
beef hamburger
hamburger patty
hamburger steak
double hamburger
ground hamburger
turkey hamburger
hamburger no bun
grilled hamburger
hamburger patties
fast food hamburger
cheeseburger macaroni hamburger helper
Hamburger
Hamburger Buns, Enriched
Buns, Hamburger
Hamburger Buns, Classic
Hamburger Buns
Hamburger Buns, Sesame
Hamburger Buns, Brioche Style
Keto Friendly Hamburger Buns
Hamburger Buns, Enriched, White
Hamburger Buns, Hamburger, Sliced
Hamburger Buns, Soft White
Keto Hamburger Buns
Hamburger Dill Chips
Hamburger Buns, Butter
White Hamburger Buns
Hamburger Stackers, Dill
Hamburger Buns, Classic White
Ovals Hamburger Dill Chips
BLT
Cheeseburger


Given the UPC code, print the food

In [10]:
upc = "49000000450"

url = "https://trackapi.nutritionix.com/v2/search/item/?upc=" + upc
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "x-app-id": app_id,
    "x-app-key": app_key
}

response = requests.get(url, headers=headers)

if response.status_code != 200:
    raise Exception(f"Error: {response.status_code}")
else:
    # print(response.json())
    for food in response.json()["foods"]:
        print(food)


{'food_name': 'Diet Cola', 'brand_name': 'Coke', 'serving_qty': 1, 'serving_unit': 'bottle', 'serving_weight_grams': 226, 'nf_metric_qty': 591, 'nf_metric_uom': 'ml', 'nf_calories': 0, 'nf_total_fat': 0, 'nf_saturated_fat': None, 'nf_cholesterol': None, 'nf_sodium': 70, 'nf_total_carbohydrate': 0, 'nf_dietary_fiber': None, 'nf_sugars': None, 'nf_protein': 0, 'nf_potassium': None, 'nf_p': None, 'full_nutrients': [{'attr_id': 203, 'value': 0}, {'attr_id': 204, 'value': 0}, {'attr_id': 205, 'value': 0}, {'attr_id': 208, 'value': 0}, {'attr_id': 307, 'value': 70}], 'nix_brand_name': 'Coke', 'nix_brand_id': '51db3801176fe9790a89ae0b', 'nix_item_name': 'Diet Cola', 'nix_item_id': '51d2fae7cc9bff111580d8d8', 'metadata': {}, 'source': 8, 'ndb_no': None, 'alt_measures': None, 'lat': None, 'lng': None, 'photo': {'thumb': 'https://nutritionix-api.s3.amazonaws.com/5b5c17c429f162275dc922b0.jpeg', 'highres': None, 'is_user_uploaded': False}, 'note': None, 'class_code': None, 'brick_code': None, 'tag

Given the amount of exercise, print the exercise information

In [11]:
url = 'https://trackapi.nutritionix.com/v2/natural/exercise'
headers = {
    'Content-Type': 'application/json',
    'x-app-id': app_id,
    'x-app-key': app_key
}
body = {
    "query": "swam for 1 hour"
}

response = requests.post(url, headers=headers, data=json.dumps(body))

if response.status_code != 200:
    raise Exception(f"Error: {response.status_code}")
else:
    print(response.json())


{'exercises': [{'tag_id': 63, 'user_input': 'swam', 'duration_min': 60, 'met': 6, 'nf_calories': 420, 'photo': {'highres': 'https://d2xdmhkmkbyw75.cloudfront.net/exercise//63_highres.jpg', 'thumb': 'https://d2xdmhkmkbyw75.cloudfront.net/exercise//63_thumb.jpg', 'is_user_uploaded': False}, 'compendium_code': 18310, 'name': 'swimming', 'description': None, 'benefits': None}]}
